### Training

In [ ]:
%pip install ultralytics

In [ ]:
import os, yaml

# === 1) Khai báo đúng 10 lớp (theo mapping bạn đã convert) ===
class_names = [
    'person', 'rider', 'car', 'bus', 'truck',
    'bike', 'motor', 'traffic light', 'traffic sign', 'train'
]

# === 2) Tạo data.yaml đúng với cây thư mục hiện tại ===
ROOT = r"C:\Users\anhhu\Downloads\Company Internship\BDD100k"   # thư mục chứa images/ và labels/
yaml_file = r"C:\Users\anhhu\Downloads\Company Internship\bdd100k.yaml"

data = {
    "path": ROOT.replace("\\", "/"),   # dùng / cho chắc ăn trên Windows
    "train": "images/train",
    "val": "images/val",
    "test": "images/test",             # có cũng được, không có YOLO sẽ bỏ qua
    "names": class_names
}

os.makedirs(os.path.dirname(yaml_file), exist_ok=True)
with open(yaml_file, "w", encoding="utf-8") as f:
    yaml.dump(data, f, sort_keys=False, allow_unicode=True)

print(f"YOLO config saved to: {yaml_file}")

# (khuyến nghị) sanity check nhanh để chắc thư mục tồn tại
for p in [ROOT, os.path.join(ROOT, "images/train"), os.path.join(ROOT, "images/val")]:
    print("OK" if os.path.exists(p) else "MISSING", "->", p)


In [ ]:
# compare_yolov9_only.py
from ultralytics import YOLO
import os, yaml, torch, random, time, csv
from pathlib import Path

# =============== CẤU HÌNH CỐ ĐỊNH ===============
YAML_FILE = r"C:\Users\anhhu\Downloads\Company Internship\bdd100k.yaml"
SAVE_ROOT = r"C:\Users\anhhu\runs\detect_compare_v9"
EPOCHS    = 50 
IMG_SIZE  = 640
BATCH     = 64            # nếu OOM, giảm đều cho tất cả model trong danh sách // chuyển sang  64 để bằng v8v8
SEED      = 42
WORKERS   = 4          # Windows => 0 , chuyển sang 4
CLOSE_MOSAIC = 10
DETERMINISTIC = True
CACHE     = True        # CHuyển sang True

# CHỈ YOLOv9 — thêm/bớt biến thể ở đây
MODELS = [
    ("yolov9t.pt", "v9t_bdd100k"),
    # ("yolov9s.pt", "v9s_bdd100k"),
    # ("yolov9m.pt", "v9m_bdd100k"),
    # ("yolov9c.pt", "v9c_bdd100k"),
    # ("yolov9e.pt", "v9e_bdd100k"),
]

# =============== TIỆN ÍCH ===============
def set_reproducible(seed: int = 42, deterministic: bool = True):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

def sanity_check_yaml(path_yaml: str):
    assert os.path.exists(path_yaml), f"Không thấy YAML: {path_yaml}"
    with open(path_yaml, "r", encoding="utf-8") as f:
        data = yaml.safe_load(f)
    print("=== DATA.YAML ===")
    for k in ["path", "train", "val", "test", "names"]:
        if k in data:
            print(f"{k}: {data[k]}")
    print("=================")

def model_params(model):
    return sum(p.numel() for p in model.model.parameters())

def benchmark_speed(model, device, imgs=20, imgsz=IMG_SIZE):
    model.to(device)
    model.model.eval()
    x = torch.randn(1, 3, imgsz, imgsz, device=device)
    # warmup
    for _ in range(5):
        with torch.inference_mode():
            _ = model.model(x)
    if device != "cpu" and torch.cuda.is_available():
        torch.cuda.synchronize()
    t0 = time.time()
    with torch.inference_mode():
        for _ in range(imgs):
            _ = model.model(x)
    if device != "cpu" and torch.cuda.is_available():
        torch.cuda.synchronize()
    return (time.time() - t0) / imgs  # giây/ảnh

def safe_train(model, device_arg, run_name):
    try:
        results = model.train(
            data=YAML_FILE,
            epochs=EPOCHS,
            imgsz=IMG_SIZE,
            batch=BATCH,
            workers=WORKERS,
            device=device_arg,
            cache=CACHE,
            deterministic=DETERMINISTIC,
            close_mosaic=CLOSE_MOSAIC,
            pretrained=True,
            project=SAVE_ROOT,
            name=run_name,
            save_period=5,
            patience=0,
            seed=42,        # <-- thêm: cố định seed cho fair compare
            amp=False       # <-- thêm: tắt AMP hẳn trên GTX 1650 Ti
        )
        return results
    except RuntimeError as e:
        if "out of memory" in str(e).lower():
            raise RuntimeError(
                f"OOM khi batch={BATCH}. Hãy giảm BATCH (vd 4 hoặc 2) cho TẤT CẢ model trong MODELS để giữ công bằng."
            )
        raise

# =============== MAIN ===============
def main():
    os.makedirs(SAVE_ROOT, exist_ok=True)
    set_reproducible(SEED, DETERMINISTIC)
    sanity_check_yaml(YAML_FILE)

    device_arg = 0 if torch.cuda.is_available() else "cpu"
    print("CUDA available:", torch.cuda.is_available())
    print(f"Using device: {device_arg}")

    csv_path = os.path.join(SAVE_ROOT, "summary_v9.csv")
    with open(csv_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([
            "model", "run_name", "epochs", "imgsz", "batch",
            "mAP50-95", "mAP50", "mAP75",
            "precision", "recall",
            "params",
            "avg_infer_sec_per_img",
            "save_dir"
        ])

        for weights, run_name in MODELS:
            print(f"\n========== TRAIN & EVAL: {weights} ==========")
            model = YOLO(weights)
            model.info()  # in kiến trúc/tổng quan

            # Train
            results = safe_train(model, device_arg, run_name)

            # Val
            metrics = model.val(
                data=YAML_FILE,
                imgsz=IMG_SIZE,
                batch=BATCH,
                device=device_arg,
                plots=False,
                save_json=False
            )

            # Lấy chỉ số
            mAP5095 = float(getattr(metrics.box, "map", 0.0))     # mAP@0.5:0.95
            mAP50   = float(getattr(metrics.box, "map50", 0.0))
            mAP75   = float(getattr(metrics.box, "map75", 0.0))
            prec    = float(getattr(metrics.box, "mp", 0.0))
            rec     = float(getattr(metrics.box, "mr", 0.0))

            params  = model_params(model)
            avg_infer = benchmark_speed(model, device="cuda:0" if device_arg==0 else "cpu",
                                        imgs=20, imgsz=IMG_SIZE)

            try:
                save_dir = str(results.save_dir)
            except Exception:
                save_dir = getattr(getattr(model, "trainer", None), "save_dir", "")

            writer.writerow([
                weights, run_name, EPOCHS, IMG_SIZE, BATCH,
                round(mAP5095, 4), round(mAP50, 4), round(mAP75, 4),
                round(prec, 4), round(rec, 4),
                params,
                round(avg_infer, 5),
                save_dir
            ])
            print(f"→ Saved row for {weights} at {csv_path}")

    print("\n Hoàn tất. Mở file summary_v9.csv để xem bảng so sánh các biến thể YOLOv9.")

if __name__ == "__main__":
    main()
